[View in Colaboratory](https://colab.research.google.com/github/paulanavarretec/RecSys-Tarea1/blob/master/Recsys_Tarea1.ipynb)

# Recommender Algorithms for a Beer Selling platform


---



In this asisgnment I develop several Recommender algorithms and test their performance. I use pyreclab Python library, developed by IALab & SocVis from the Pontifical Catholic University of Chile.

The algorithms developed ahead are:

> UserKnn

> ItemKnn

> SlopeOne

> SVD


> ALS o ALScg: (este modelo lo puede usar para la tarea de ranking. Si tiene problemas con pyreclab, puede usar la biblioteca implicit1).




## Paso 1: Descargue los siguientes tres archivos, siguiendo los links, a su computador local (u1.base, u1.test y u.item, respectivamente)


---

